<a href="https://colab.research.google.com/github/bnnguyen/DESLab_ML_training_2024/blob/main/Deslab_2024_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project mẫu - thăm dò bộ dữ liệu cuộc gọi khẩn cấp

Đối với dự án này, chúng ta sẽ phân tích một số dữ liệu cuộc gọi 911 từ [Kaggle](https://www.kaggle.com/mchirico/montcoalert). Dữ liệu chứa các trường sau:

* lat : Biến chuỗi, Vĩ độ
* lng: Biến chuỗi, Kinh độ
* desc: Biến chuỗi, Mô tả cuộc gọi khẩn cấp
* zip: Biến chuỗi, Zipcode
* title: Biến chuỗi, Tiêu đề
* timeStamp: Biến chuỗi, YYYY-MM-DD HH:MM:SS
* twp: Biến chuỗi, Thị trấn
* addr: Biến chuỗi, Địa chỉ
* e: Biến chuỗi, biến giả (luôn luôn là 1)

## Dữ liệu và thiết lập cơ bản

____
**Import numpy và pandas**

In [ ]:
import numpy as np
import pandas as pd
import requests
import io

**Import thư viện trực quan hóa và đặt %matplotlib inline.**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid') #whitegrid

**Đọc tệp csv dưới dạng khung dữ liệu có tên df**

In [ ]:
url = "https://raw.githubusercontent.com/bchoivw/python-data-science-study/master/911-eda/911.csv"
df = pd.read_csv(url)

**Dùng hàm info() để kiểm tra df**

In [ ]:
df.info()

**Kiểm tra các biến ở đầu của df**

In [ ]:
df.head()

## Câu hỏi cơ bản cần nắm bắt

**5 mã zip hàng đầu cho cuộc gọi 911 là?**

In [ ]:
df['zip'].value_counts().head(5)

**5 thị trấn (twp) có nhiều cuộc gọi 911 nhất là?**

In [ ]:
df['twp'].value_counts().head(5)

**Có bao nhiêu loại tiêu đề khác nhau?**

In [ ]:
df["title"].nunique()

## Tạo feature mới

**Tại cột titles có ghi rõ "Reasons/Departments" trước mã title. Các loại "Reasons/Departments" là EMS, Fire, và Traffic. Ở đây, chúng ta sử dụng .apply() với biểu thức lambda tùy chỉnh để tạo cột mới có tên "Reason" chứa giá trị chuỗi này.**

**Ví dụ: nếu giá trị cột tiêu đề là EMS: BACK PAINS/INJURY , thì giá trị cột Lý do sẽ là EMS.**

In [ ]:
df['Reason'] = df['title'].apply(lambda x: x.split(':')[0])

In [ ]:
df.head()

**Lý do phổ biến nhất dẫn đến cuộc gọi 911 (dựa trên cột mới này)?**

In [ ]:
df['Reason'].value_counts()

**Bây giờ, hãy sử dụng seaborn để tạo bảng đếm các cuộc gọi 911 theo Reason.**

In [ ]:
sns.countplot(x="Reason", data=df)

___
**Bây giờ chúng ta hãy bắt đầu tập trung vào thông tin về thời gian. Kiểu dữ liệu của đối tượng trong cột timeStamp là gì?**

In [ ]:
type(df['timeStamp'].iloc[0])

**Bạn hẳn đã thấy rằng những dấu thời gian này vẫn là chuỗi. Chúng ta sẽ sử dụng [pd.to_datetime](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html) để chuyển đổi cột từ chuỗi thành đối tượng DateTime.**

In [ ]:
df['timeStamp'] = pd.to_datetime(df['timeStamp'])

**Bây giờ bạn có thể lấy các thuộc tính cụ thể từ một đối tượng Datetime bằng cách gọi chúng. Ví dụ:**

    time = df['timeStamp'].iloc[0]
    time.hour

In [ ]:
df.head()

In [ ]:
df['Hour'] = df['timeStamp'].apply(lambda time: time.hour)
df['Month'] = df['timeStamp'].apply(lambda time: time.month)
df['Day of Week'] = df['timeStamp'].apply(lambda time: time.dayofweek)

In [ ]:
df.head(3)

**Lưu ý: Day of week là số nguyên 0-6. Chúng ta sẽ sử dụng .map() với từ điển này để ánh xạ tên chuỗi thực tế theo ngày trong tuần:**

    dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
dmap = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}

In [ ]:
df['Day of Week'] = df['Day of Week'].map(dmap)

In [ ]:
df.head(2)

**Bây giờ, hãy sử dụng seaborn để tạo biểu đồ đếm của cột Day of week với màu sắc dựa trên cột Reason.**

In [ ]:
sns.countplot(x='Day of Week', data=df, hue='Reason')
plt.legend(bbox_to_anchor=(1.05,1), loc = 2, borderaxespad=0.)

**Làm tương tự với Month:**

In [ ]:
sns.countplot(x='Month', data=df, hue='Reason')
plt.legend(bbox_to_anchor=(1.05,1), loc = 2, borderaxespad=0.)

**Bạn có nhận thấy điều gì 'hơi lạ' ở các đồ thị trên hong?**





**Bây giờ hãy tạo một đối tượng nhóm có tên byMonth, trong đó bạn nhóm DataFrame theo cột tháng và sử dụng phương thức count() để tổng hợp. Sử dụng phương thức head() trên DataFrame được trả về:**

In [ ]:
byMonth = df.groupby("Month").count()
byMonth.head(12)

**Bây giờ hãy tạo một biểu đồ đơn giản từ khung dữ liệu cho biết số lượng cuộc gọi mỗi tháng.**

In [ ]:
byMonth['e'].plot()

**Bây giờ hãy xem liệu bạn có thể sử dụng lmplot() của seaborn để tạo sự phù hợp tuyến tính về số lượng cuộc gọi mỗi tháng hay không.**

In [ ]:
# Look up .reset_index
sns.lmplot(x='Month', y='e', data=byMonth.reset_index())

**Ở đây, chúng ta tạo một cột mới có tên là'Date' chứa ngày từ cột timeStamp. Chúng ta sẽ cần sử dụng apply cùng với .date().**

In [ ]:
df['Date'] = df['timeStamp'].apply(lambda time: time.date())
df.head()

**Bây giờ, hãy groupby theo cột Ngày này với tổng số count() và tạo biểu đồ đếm số cuộc gọi 911.**

In [ ]:
df.groupby('Date').count()['e'].plot(figsize=(10,6))

**Bây giờ hãy tạo lại biểu đồ này nhưng tạo 3 biểu đồ riêng biệt với mỗi biểu đồ đại diện cho một lý do gọi 911**

In [ ]:
#Reason 1: EMS
df[df['Reason']=='EMS'].groupby('Date').count()['e'].plot(figsize=(10,6))

In [ ]:
#Reason 2: Fire
df[df['Reason']=='Fire'].groupby('Date').count()['e'].plot(figsize=(10,6))

In [ ]:
#Reason 3: Traffic
df[df['Reason']=='Traffic'].groupby('Date').count()['e'].plot(figsize=(10,6))

____
**Bây giờ chúng ta sẽ chuyển sang tạo heatmap với seaborn và dữ liệu của chúng ta. Trước tiên, chúng ta cần cơ cấu lại khung dữ liệu để các cột Hours và Index trở thành Day of Week.**

In [ ]:
hdf = df.groupby(['Day of Week','Hour']).count()['e'].unstack(1)

In [ ]:
hdf

**Giờ thì tạo heatmap thôi nào.**



In [ ]:
plt.figure(figsize=(12,6))
sns.heatmap(data=hdf)


**Chúng ta cũng có thể tạo clustermap dựa vào hdf. Tài liệu: https://seaborn.pydata.org/generated/seaborn.clustermap.html**

In [ ]:
sns.clustermap(hdf)


**Bây giờ, hãy lặp lại các biểu đồ tương tự (heatmap và clustermap) cho DataFrame hiển thị Month dưới dạng cột**

In [ ]:
dayMonth = df.groupby(by=['Day of Week','Month']).count()['Reason'].unstack()
dayMonth.head()

In [ ]:
#ClusterMap
sns.clustermap(dayMonth)

In [ ]:
#HeatMap
plt.figure(figsize=(12,6))
sns.heatmap(dayMonth)